In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, metrics, preprocessing
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [17]:
class BookRecommender(nn.Module):
    def __init__(self, n_books, n_users):
        super().__init__()
        
        self.user_embed = nn.Embedding(n_users, 32)
        self.book_embed = nn.Embedding(n_books, 32)
        self.out = nn.Linear(64, 1)


    def forward(self, users, books):
        user_embed = self.user_embed(users)
        book_embed = self.book_embed(books)
        output = torch.cat([user_embed, book_embed], dim=1) 
        output = self.out(output)

        return output

In [51]:
ratings_df = pd.read_csv('main_dataset/updated_ratings.csv')

lbl_userid = preprocessing.LabelEncoder()
lbl_isbn = preprocessing.LabelEncoder()
ratings_df["user_id"] = lbl_userid.fit_transform(ratings_df["user_id"].values)
ratings_df["isbn"] = lbl_isbn.fit_transform(ratings_df["isbn"].values)
ratings_df = ratings_df[ratings_df["rating"] != 0]

ratings_df

,user_id,isbn,rating
6,53992,40226,4.0
7,53993,43448,3.0
8,53994,40924,4.0
9,53995,28356,2.5
11,53996,17298,2.5
...,...,...,...
467083,53985,51274,5.0
467084,53985,52819,3.0
467094,53987,10758,3.0
467098,53988,31174,5.0


In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [65]:
ratings_df[ratings_df["user_id"].isin([1343, 54420, 39])]
# sorted(ratings_df["user_id"].unique())

,user_id,isbn,rating
3178,54420,8885,4.0
3282,39,3989,3.0
11459,1343,22008,3.5
11460,1343,34303,4.0


In [68]:
ratings_df.head(3)

,user_id,isbn,rating
6,53992,40226,4.0
7,53993,43448,3.0
8,53994,40924,4.0


In [71]:
model = BookRecommender(58476, 54426)
model.load_state_dict(torch.load('./models/model_cross_val.pth'))

model.eval()
with torch.no_grad():
    # Convert user IDs to embeddings
    user_embeddings = model.user_embed(torch.tensor([1343, 54420, 39], dtype=torch.long))
    print(user_embeddings.shape)

    # Convert book IDs to embeddings
    book_embeddings = model.book_embed(torch.tensor([8885, 3989, 34303], dtype=torch.long))
    print(book_embeddings.shape)

    # Concatenate user and book embeddings along dimension 2
    output = torch.cat([user_embeddings, book_embeddings], dim=1)
    print(output.shape)

    # Pass the concatenated embeddings through the model
    output = model.out(output)
    expectation = ratings_df.loc[ratings_df["user_id"].isin([1343, 54420, 39])]

    # Print the output    
    print(f"\n\nModel Prediction:\n{output}")
    print(f"\n\nExpected result:\n{expectation}")

torch.Size([3, 32])
torch.Size([3, 32])
torch.Size([3, 64])


Model Prediction:
tensor([[4.1085],
        [3.7871],
        [3.0093]])


Expected result:
       user_id   isbn  rating
3178     54420   8885     4.0
3282        39   3989     3.0
11459     1343  22008     3.5
11460     1343  34303     4.0
